# Fuentes de datos

Disponemos de varias opciones para el acceso a fuentes de datos. Existen dos tipologías principales cuando hablamos de acceso a datos:

* APIs REST: Servicios a los que podemos consultar una foto puntual de los datos.
* Bases de datos: Sistemas de gestión de datos a los que podemos interrogar sobre su contenido

## API

Las APIs (Application Programming Interfaces) permiten la comunicación entre diferentes aplicaciones a través de peticiones HTTP. Son muy utilizadas para acceder a datos de servicios externos de forma estructurada y segura.

En Python, el consumo de APIs suele realizarse mediante la librería `requests`, que facilita el envío de peticiones y la gestión de las respuestas.

In [1]:
import pprint
import requests

url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    pprint.pprint(data)
else:
    print("Error al consultar la API:", response.status_code)

{'body': 'quia et suscipit\n'
         'suscipit recusandae consequuntur expedita et cum\n'
         'reprehenderit molestiae ut ut quas totam\n'
         'nostrum rerum est autem sunt rem eveniet architecto',
 'id': 1,
 'title': 'sunt aut facere repellat provident occaecati excepturi optio '
          'reprehenderit',
 'userId': 1}


Este código realiza una petición GET al endpoint y muestra el contenido recibido en formato JSON. La librería request ofrece todas las opciones de un protocolo HTTP incluyendo opciones de autenticación mediante cabeceras.

### Consumo continuado

Cuando se trata de una fuente de la que debamos consumir con cierta frecuencia, es recomendable disponer de un proceso de extracción y carga (EL). Librerías específicas como [dlt](https://dlthub.com/) (data load tool) son muy útiles para aquellos que no dispongamos de plataformas de ELT/ETL corporativas y trabajemos en entornos Python.

Podemos user como ejemplo la información de https://thespacedevs.com/ sobre lanzamientos espaciales.

* Producción (con limitaciones) https://ll.thespacedevs.com/docs 
* Desarrollo https://lldev.thespacedevs.com/docs
* Consulta de peticiones restantes https://ll.thespacedevs.com/2.3.0/api-throttle/ 

In [2]:
space_url = "http://lldev.thespacedevs.com/2.2.0"

# Endpoint de astronautas
path = "/astronaut"
url_total = space_url + path

# Petición GET
response = requests.get(url_total)

In [3]:
import pandas as pd

respuesta = response.json()
pd.DataFrame(respuesta['results'][:3])

,id,url,name,status,type,in_space,time_in_space,eva_time,age,date_of_birth,...,instagram,wiki,agency,profile_image,profile_image_thumbnail,flights_count,landings_count,spacewalks_count,last_flight,first_flight
0,1,https://lldev.thespacedevs.com/2.2.0/astronaut/1/,Thomas Pesquet,"{'id': 1, 'name': 'Active'}","{'id': 2, 'name': 'Government'}",False,P396DT11H33M45S,P1DT15H54M,47,1978-02-27,...,https://instagram.com/thom_astro,https://en.wikipedia.org/wiki/Thomas_Pesquet,"{'id': 27, 'url': 'https://lldev.thespacedevs....",https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,2,2,6,2021-04-23T09:49:02Z,2016-11-17T20:20:13Z
1,2,https://lldev.thespacedevs.com/2.2.0/astronaut/2/,Claude Nicollier,"{'id': 2, 'name': 'Retired'}","{'id': 2, 'name': 'Government'}",False,P42DT12H3M12S,PT8H10M,80,1944-09-02,...,None,https://en.wikipedia.org/wiki/Claude_Nicollier,"{'id': 27, 'url': 'https://lldev.thespacedevs....",https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,4,4,1,1999-12-20T00:50:00Z,1992-07-31T13:56:48Z
2,3,https://lldev.thespacedevs.com/2.2.0/astronaut/3/,Tim Peake,"{'id': 2, 'name': 'Retired'}","{'id': 2, 'name': 'Government'}",False,P185DT22H11M51S,PT4H43M,53,1972-04-07,...,https://www.instagram.com/astro_timpeake/,https://en.wikipedia.org/wiki/Tim_Peake,"{'id': 27, 'url': 'https://lldev.thespacedevs....",https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,1,1,1,2015-12-15T11:03:09Z,2015-12-15T11:03:09Z


Si quisiéramos hacer esto de forma sistemática y almacenarlo en una base de datos para su consulta posterior, podemos usar dlt y almacenarlo en una base de datos (local para estos ejercicios).

In [4]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import JSONLinkPaginator

# Cliente REST
client = RESTClient(
    base_url=space_url,
    paginator=JSONLinkPaginator(next_url_path="next"),
    data_selector="results"
)

# Resource (con identificaciones sobre tipo)
@dlt.resource(columns={'agency__parent': {'data_type': 'text'}, 'agency' : {'data_type': 'text'}})
def astronauts():
    for page in client.paginate(
        path,
        params={
            "limit": 100,
        },
    ):
        yield page

In [5]:
# Pipeline
pipeline = dlt.pipeline(
    pipeline_name="space_data",
    destination="duckdb",
    dataset_name="space",
    progress='log'
)

load_info = pipeline.run(astronauts)

------------------------------ Extract space_data ------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 173.68 MB (44.50%) | CPU usage: 0.00%

------------------------------ Extract space_data ------------------------------
Resources: 0/1 (0.0%) | Time: 0.60s | Rate: 0.00/s
astronauts: 100  | Time: 0.00s | Rate: 24672376.47/s
Memory usage: 175.40 MB (44.40%) | CPU usage: 0.00%

------------------------------ Extract space_data ------------------------------
Resources: 0/1 (0.0%) | Time: 1.71s | Rate: 0.00/s
astronauts: 700  | Time: 1.10s | Rate: 634.20/s
Memory usage: 179.30 MB (44.40%) | CPU usage: 0.00%

------------------------------ Extract space_data ------------------------------
Resources: 1/1 (100.0%) | Time: 2.14s | Rate: 0.47/s
astronauts: 819  | Time: 1.53s | Rate: 534.71/s
Memory usage: 180.06 MB (44.20%) | CPU usage: 0.00%

------------------------------ Extract space_data ------------------------------
Resources: 0/1 (0.0%) | Time

In [6]:
print(load_info)

Pipeline space_data load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset space
The duckdb destination used duckdb:////home/iraitz/TheBridge/B2B/DS4B2B/M1 - Exploración de datos/space_data.duckdb location to store data
Load package 1753531284.8009744 is LOADED and contains no failed jobs


En este caso hemos optado por almacenar la información en una base de datos local [DuckDB](https://duckdb.org/) a la que podemos consultar por la información recibida.

In [1]:
import duckdb

db = duckdb.connect(database="space_data.duckdb")
db.sql("DESCRIBE;")

┌────────────┬─────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Tenemos información de metadatos sobre la carga realizada en DLT (tiempos, hash de los elementos, etc.) muy útiles cuando queremos realizar estas cargas de forma periódica. Podemos una vez cargado en la base de datos, recurrir a Pandas para consultar las tablas.

In [2]:
response = db.sql("SELECT name, flights_count, landings_count, spacewalks_count FROM space.astronauts WHERE in_space == True;")
response.to_df()

,name,flights_count,landings_count,spacewalks_count
0,Anne McClain,2,1,3
1,Chen Dong,3,2,5
2,Takuya Onishi,2,1,0
3,Jonny Kim,1,0,0
4,Sergey Ryzhikov,3,2,1
5,Starman,1,0,0
6,Nichole Ayers,1,0,1
7,Kirill Peskov,1,0,0
8,Alexey Zubritsky,1,0,0
9,Zhongrui Chen,1,0,2


In [3]:
db.close()

#### ETL vs ELT

En los últimos años se han priorizado distintas modalidades sobre dónde suceden las transformaciones.

* **ETL**: Los procesos ETL nos obligan a disponer uns infraestructura intermedia encargada de realizar las transformaciones
* **ELT** : Los procesos de extracción y carga ([dlt](https://dlthub.com/)) seguidos de procesos de transformación que explotan la naturaleza declarativa de SQL ([dbt](https://www.getdbt.com/)) nos ofrecen una mayor flexibilidad y aprovechamiento de las infraestructuras corporativas aunque plantean un almacenamiento flexible para poder albergar lo que venga de origen.

![](https://learn.microsoft.com/es-es/azure/databricks/_static/images/lakehouse-architecture/medallion-architecture.png)

Más sobre la [Medallion Architecture](https://learn.microsoft.com/es-es/azure/databricks/lakehouse/medallion).

## Base de datos

Pandas nos ofrece poder leer información de fuentes estructuradas como son las bases de datos. Para esto se apoya en la librería [SQLAlchemy](https://www.sqlalchemy.org/) para poder comunicarse mediante el protocolo necesario para la base de datos en cuestión.

In [4]:
import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine('duckdb:///space_data.duckdb')
con = engine.connect()

pd.read_sql_query("SELECT name, flights_count, landings_count, spacewalks_count FROM space.astronauts WHERE in_space == True;",con=con)

,name,flights_count,landings_count,spacewalks_count
0,Anne McClain,2,1,3
1,Chen Dong,3,2,5
2,Takuya Onishi,2,1,0
3,Jonny Kim,1,0,0
4,Sergey Ryzhikov,3,2,1
5,Starman,1,0,0
6,Nichole Ayers,1,0,1
7,Kirill Peskov,1,0,0
8,Alexey Zubritsky,1,0,0
9,Zhongrui Chen,1,0,2


In [5]:
con.close()

Debido a que nos encontramos ante un recurso que ofrece capacidad de computo por si mismo, pero queremos seguir empleando la sintaxis de pandas (al menos para los científicos de datos acostumbrados a esta), existen motores de consulta que nos permiten implementar la sintaxis de esta manera pero delegar el computo a la base de datos (es decir, lanzando una serie de consultas).

#### Ibis

[Ibis](https://ibis-project.org/) es un motor de uso general que nos ofrece estas capacidades sobre una cantidad importante de _backends_

![](../assets/images/ibis_back.png)

In [ ]:
import ibis

con = ibis.connect("duckdb://space_data.duckdb")
astronauts = con.sql("SELECT * FROM space.astronauts")

Ibis no ejecuta nada, si no que declara qué es la estructura astronauts, de modo que podemos ir pidiendo acciones sobre esa estructura como si de un DataFrame se tratara. Por ejemplo, si ejecutamos `head()` veremos que nos muestra cómo añade una sentencia limit al final de nuestra estructura.

In [6]:
astronauts.head()

r0 := SQLQueryResult
  query:
    SELECT * FROM space.astronauts
  schema:
    agency__parent          string
    agency                  string
    id                      int64
    url                     string
    name                    string
    status__id              int64
    status__name            string
    type__id                int64
    type__name              string
    in_space                boolean
    time_in_space           string
    eva_time                string
    age                     int64
    date_of_birth           string
    nationality             string
    bio                     string
    twitter                 string
    instagram               string
    wiki                    string
    agency__id              int64
    agency__url             string
    agency__name            string
    agency__featured        boolean
    agency__type            string
    agency__country_code    string
    agency__abbrev          string
    agency__description     string
    agency__administrator   string
    agency__founding_year   string
    agency__launchers       string
    agency__spacecraft      string
    agency__image_url       string
    agency__logo_url        string
    profile_image           string
    profile_image_thumbnail string
    flights_count           int64
    landings_count          int64
    spacewalks_count        int64
    last_flight             timestamp('UTC', 6)
    first_flight            timestamp('UTC', 6)
    _dlt_load_id            !string
    _dlt_id                 !string
    date_of_death           string

Limit[r0, n=5]

Podemos pedir que lo transforme en un objeto pandas, que ejecuta de forma efectiva nuestra consulta.

In [8]:
astronauts.head().to_pandas()

,agency__parent,agency,id,url,name,status__id,status__name,type__id,type__name,in_space,...,profile_image,profile_image_thumbnail,flights_count,landings_count,spacewalks_count,last_flight,first_flight,_dlt_load_id,_dlt_id,date_of_death
0,None,None,1,https://lldev.thespacedevs.com/2.2.0/astronaut/1/,Thomas Pesquet,1,Active,2,Government,False,...,https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,2,2,6,2021-04-23 09:49:02+00:00,2016-11-17 20:20:13+00:00,1753531284.8009744,kZYm9TbyVcS6Tw,None
1,None,None,2,https://lldev.thespacedevs.com/2.2.0/astronaut/2/,Claude Nicollier,2,Retired,2,Government,False,...,https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,4,4,1,1999-12-20 00:50:00+00:00,1992-07-31 13:56:48+00:00,1753531284.8009744,6mtA3IiKgj//vw,None
2,None,None,3,https://lldev.thespacedevs.com/2.2.0/astronaut/3/,Tim Peake,2,Retired,2,Government,False,...,https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,1,1,1,2015-12-15 11:03:09+00:00,2015-12-15 11:03:09+00:00,1753531284.8009744,39MxVluP83YQ9Q,None
3,None,None,4,https://lldev.thespacedevs.com/2.2.0/astronaut/4/,Buzz Aldrin,2,Retired,2,Government,False,...,https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,2,2,4,1969-07-21 17:54:00+00:00,1966-11-11 20:46:33+00:00,1753531284.8009744,9vGS6dh1JOw4MQ,None
4,None,None,5,https://lldev.thespacedevs.com/2.2.0/astronaut/5/,Chris Hadfield,2,Retired,2,Government,False,...,https://thespacedevs-dev.nyc3.digitaloceanspac...,https://thespacedevs-dev.nyc3.digitaloceanspac...,3,3,2,2012-12-19 12:12:35+00:00,1995-11-12 12:30:43+00:00,1753531284.8009744,to93u/Yytbb0fA,None


Y también podremos ver la sentencia que resulta.

In [11]:
ibis.to_sql(astronauts)

```sql
SELECT
  *
FROM space.astronauts
```

In [12]:
ibis.to_sql(astronauts.head())

```sql
SELECT
  *
FROM space.astronauts
LIMIT 5
```

Esto permite a los Data Scientist componer las consultas como si de DataFrames se tratara y solo ejecutar las sentencias en base de datos cuando de forma efectiva se quieren observar los resultados.

In [14]:
consulta = astronauts.filter(astronauts.in_space == True).select("name", "flights_count", "landings_count", "spacewalks_count")
consulta.to_pandas()

,name,flights_count,landings_count,spacewalks_count
0,Anne McClain,2,1,3
1,Chen Dong,3,2,5
2,Takuya Onishi,2,1,0
3,Jonny Kim,1,0,0
4,Sergey Ryzhikov,3,2,1
5,Starman,1,0,0
6,Nichole Ayers,1,0,1
7,Kirill Peskov,1,0,0
8,Alexey Zubritsky,1,0,0
9,Zhongrui Chen,1,0,2


In [15]:
ibis.to_sql(consulta)

```sql
SELECT
  "t0"."name",
  "t0"."flights_count",
  "t0"."landings_count",
  "t0"."spacewalks_count"
FROM (
  SELECT
    *
  FROM space.astronauts
) AS "t0"
WHERE
  "t0"."in_space" = TRUE
```